# Restore results from pickle to test OpenMSI store

In [1]:
import os
from omaat_lib import ArrayedImage, login

In [2]:
#log into OpenMSI.nersc.gov
openMSIsession = login()

Login appears to be successful!


In [3]:
# Load an ArrayedImage from a pickle file. 
filename="filename.arrayed_img"
import pickle
img=pickle.load(open(filename,"rb"))

In [4]:
%store -r arrayed_analysis_radius
spots=img.generateSpotList(integrationRadius=arrayed_analysis_radius)

4 spots generated. number of spots with N pixels:{11: 1, 13: 3}


In [5]:
spectra_df=openMSIsession.getSpotSpectra(img,verbose=True)

Finished loading spectrum 1 out of 4
Finished loading spectrum 2 out of 4
Finished loading spectrum 3 out of 4
Finished loading spectrum 4 out of 4


<h2>Saving, Restoring, Uploading, and Downloading OMAAT results to/from OpenMSI</h2>

You can save the results from the spot analysis to an OpenMSI HDF5 file and upload the file to OpenMSI to easily share results with others. This functionality is available in a the ```omaat_openmsi_store.py``` module. Several of the functions depend on BASTet <a href='https://github.com/biorack/BASTet'>https://github.com/biorack/BASTet</a> and are currently Python 2 only.

In [6]:
import omaat_openmsi_store

If you only want to save Arrayed Image to an OpenMSI HDF5 file then do:

In [7]:
# Save the ArrayedImage to an OpenMSI File
save_filename = 'save_omaat8.h5'
omsi_out_file = omaat_openmsi_store.save_arrayed_image_to_omsi_file(arrayedImage=img, 
                                                                    filename=save_filename, 
                                                                    spotSpectra=spectra_df)  
# filename : If None, than a dialog will be shown to ask for a filename to be used

We can now easily restore our analysis from the local file at any time via:

In [8]:
spotSpectraResored, arrayedImageRestored = omaat_openmsi_store.restore_omaat_results( 
                            openmsiSession=openMSIsession,
                            filename=save_filename,
                            localFile=True)
print arrayedImageRestored

ArrayedImage based on 20120913_nimzyme.h5
Ions loaded: [1141.35, 1143.05, 1241.25]
# of spot locations defined: 4
# of spot pixel masks defined: 4


To upload our ArrayedImage to OpenMSI we simply call:

In [10]:
omaat_openmsi_store.upload_omaat_results(username=None, filename=save_filename, machine='edison')
# filename: If we set the fileName to None, then we'll ask the user for a filename via a dialog
# username: We can set the username or we'll ask for it
# session: The NERSC NEWT session to be used (not the OpenMSI seesion). We'll create it if not given
# machine: The NERSC machine weshould use for the upload, e.g., 'cori' or 'edison'

(True, True, False, None)

We can naturally also download the complete HDF5 file from OpenMSI simply via:

In [11]:
remote_file = os.path.join(openMSIsession.username, os.path.basename(save_filename))
download_filename = omaat_openmsi_store.download_file(openmsiSession=openMSIsession,
                                                      filename=remote_file)

We can now again restore our analysis from the downloaded file in the same way as before via:

In [12]:
spotSpectra_resored, arrayedImage_restored = omaat_openmsi_store.restore_omaat_results(filename=download_filename,
                                                                                       localFile=True)
print arrayedImage_restored

ArrayedImage based on 20120913_nimzyme.h5
Ions loaded: [1141.35, 1143.05, 1241.25]
# of spot locations defined: 4
# of spot pixel masks defined: 4


We can also restore our arrayed image and spot spectra data from the remote file stored on OpenMSI directly via

In [13]:
#remote_file = os.path.join(openMSIsession.username, os.path.basename(save_filename))
#spotSpectra_resored, arrayedImage_restored = openMSIsession.restore_omaat_results(
#                            filename=remote_file,
#                            localFile=False)